### "Loading the spark context"



In [2]:
#import findspark
import pyspark
sc.stop()
#findspark.init()
#findspark.find()
#conf = pyspark.SparkConf().setMaster('spark://172.31.21.155:7077/').setAppName("sandbox")
#sc = pyspark.SparkContext(conf)


sc = pyspark.SparkContext(appName="spark-project")
#conf = pyspark.SparkConf()
#conf.setMaster("spark://34.228.116.224:7077")
#sc.hadoopConfiguration.set("fs.s3n.awsAccessKeyId", "BLABLA")
#sc.hadoopConfiguration.set("fs.s3n.awsSecretAccessKey", "....") // can contain "/"

In [3]:
rdd = sc.textFile('s3n://spark-course-data2/gutenberg_dataset/1/8/0/1804/1804.txt')

In [4]:
rdd.take(1)

['The Project Gutenberg EBook of War and the Future, by H. G. Wells']

In [5]:
rdd.filter(lambda line: "mankind" in line).count()

34

In [7]:
rdd.filter(lambda line: "mankind" in line).saveAsTextFile("s3n://spark-course-data2/result/lesson_2-2")

In [8]:
mankindLines = sc.textFile("s3n://spark-course-data2/result/lesson_2-2")
mankindLines.count()

34

In [14]:
rdd = sc.textFile("s3n://spark-course-data2/gutenberg_dataset/1/8/*/*/*")
rdd.count()
rdd.take(1)

['The Project Gutenberg EBook of Told in a French Garden, by Mildred Aldrich']

In [16]:
rdd = sc.wholeTextFiles("s3n://spark-course-data2/gutenberg_dataset/1/8/*/*/*")
rdd.count()
rdd.take(1)

[('s3n://spark-course-data2/gutenberg_dataset/1/8/0/0/18004/18004.txt',
  'The Project Gutenberg EBook of Told in a French Garden, by Mildred Aldrich\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n\r\nTitle: Told in a French Garden\r\n       August, 1914\r\n\r\nAuthor: Mildred Aldrich\r\n\r\nRelease Date: March 16, 2006 [EBook #18004]\r\n\r\nLanguage: English\r\n\r\nCharacter set encoding: ASCII\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK TOLD IN A FRENCH GARDEN ***\r\n\r\n\r\n\r\n\r\nProduced by Audrey Longhurst, Sankar Viswanathan, and the\r\nOnline Distributed Proofreading Team at http://www.pgdp.net\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n                               TOLD IN\r\n                           A FRENCH GARDEN\r\n\r\n                            

In [18]:
keys = rdd.map( lambda x:x[0] )
keys.count()
#keys.collect()

11

In [20]:
import boto3
session = boto3.Session() # leave out the profile_name argument if you haven't defined profiles
s3 = boto3.resource('s3')
bucket = s3.Bucket('spark-course-data2')
objs = bucket.objects.filter(Prefix='gutenberg_dataset/1')
for obj in objs:  
  print(obj.key)

gutenberg_dataset/1/0/103/103.txt
gutenberg_dataset/1/0/3/1038/1038-0.txt
gutenberg_dataset/1/0/4/6/10462/10462-8.txt
gutenberg_dataset/1/0/5/8/10586/10586.txt
gutenberg_dataset/1/0/5/9/10597/10597.txt
gutenberg_dataset/1/0/6/8/10688/10688.txt
gutenberg_dataset/1/0/7/8/10780/10780.txt
gutenberg_dataset/1/0/8/4/10848/10848.txt
gutenberg_dataset/1/0/8/5/10850/10850.txt
gutenberg_dataset/1/0/8/8/10880/10880.txt
gutenberg_dataset/1/0/9/4/10941/10941-8.txt
gutenberg_dataset/1/1/0/2/11029/11029.txt
gutenberg_dataset/1/1/1/5/11157/11157.txt
gutenberg_dataset/1/1/2/1/11219/11219.txt
gutenberg_dataset/1/1/2/4/11243/11243.txt
gutenberg_dataset/1/1/2/6/11269/11269.txt
gutenberg_dataset/1/1/5/4/11549/11549.txt
gutenberg_dataset/1/1/6/7/11676/11676.txt
gutenberg_dataset/1/1/7/3/11730/11730.txt
gutenberg_dataset/1/1/7/3/11739/11739.txt
gutenberg_dataset/1/1/8/0/11808/11808-8.txt
gutenberg_dataset/1/1/8/5/11855/11855-8.txt
gutenberg_dataset/1/1/9/1191/1191.txt
gutenberg_dataset/1/1/9/1195/1195.txt
gu

In [21]:
obj = s3.Object('spark-course-data2', 'gutenberg_dataset/1/9/9/1997/1997.txt')
content = obj.get()["Body"].read().decode("utf-8")
print(content[:100])

***The Project Gutenberg Etext of Paradise, by Dante Aligheri***

Also see our other translations 
